In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 65.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
from retrieval import ReviewRetrieval  # :contentReference[oaicite:0]{index=0}
import pandas as pd

In [ ]:
# Load POS_SHORT data
df = pd.read_csv("pos_short.csv")
df = df[["cleaned_review2", "cleaned_response2"]].dropna()

In [ ]:
retriever = ReviewRetrieval(
    df=df,
    embed_col="cleaned_review2",
    reply_col="cleaned_response2",
    model_name="sentence-transformers/all-mpnet-base-v2",
    use_gpu=True
)

Loading model: sentence-transformers/all-mpnet-base-v2 ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using GPU for embeddings.
Encoding all reviews into embeddings...


Batches:   0%|          | 0/138 [00:00<?, ?it/s]

Building FAISS index...
FAISS index built. Number of items: 4402


In [ ]:
def add_retrieval_context(row):
    query = row["cleaned_review2"]
    retrieved_replies = retriever.retrieve(query, top_k=3)
    context = " ".join(retrieved_replies)
    return context

# Apply row-wise
df["retrieved_context"] = df.apply(add_retrieval_context, axis=1)

df[["cleaned_review2", "retrieved_context", "cleaned_response2"]].head()

,cleaned_review2,retrieved_context,cleaned_response2
0,It was very pleasant and wonderful stay specia...,"Dear Valued Guest, <LOCATION> Thank you for ta...","Dear Valued Guest, <LOCATION> Thank you for ta..."
1,Good experience at the JE Marriott Juhu. Servi...,"Dear Tripadvisor Member, Thank you for taking ...","Dear Tripadvisor Member, Thank you for taking ..."
2,We came in group during our vacations in June ...,"Dear Guest, Thank you for sharing your experie...","Dear Guest, Thank you for sharing your experie..."
3,The best hotel in <LOCATION> as per my opinion...,"Dear Guest, Greetings from <HOTEL_NAME> We wel...","Dear Guest, Greetings from <HOTEL_NAME> We wel..."
4,After several very disappointing hotel experie...,"Dear Guest, Thanks for choosing and staying wi...","Dear Guest, Thanks for choosing and staying wi..."


In [ ]:
df["model_input"] = (
    "review: " + df["cleaned_review2"] +
    " retrieved: " + df["retrieved_context"]
)

df["model_target"] = df["cleaned_response2"]

df[["model_input", "model_target"]].head()

,model_input,model_target
0,review: It was very pleasant and wonderful sta...,"Dear Valued Guest, <LOCATION> Thank you for ta..."
1,review: Good experience at the JE Marriott Juh...,"Dear Tripadvisor Member, Thank you for taking ..."
2,review: We came in group during our vacations ...,"Dear Guest, Thank you for sharing your experie..."
3,review: The best hotel in <LOCATION> as per my...,"Dear Guest, Greetings from <HOTEL_NAME> We wel..."
4,review: After several very disappointing hotel...,"Dear Guest, Thanks for choosing and staying wi..."


In [ ]:
# First split: Train (70%) + Temp (30%)
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42)

# Second split: Temp into Validation (15%) + Test (15%)
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42)

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

Train size: 3081
Validation size: 660
Test size: 661


In [ ]:
# Load existing test_data.csv
existing_test_df = pd.read_csv("/content/test_data.csv")

# Append new test_df into existing file
updated_test_df = pd.concat([existing_test_df, test_df], axis=0).reset_index(drop=True)

# Save back into same file
updated_test_df.to_csv("/content/test_data.csv", index=False)

print("Appended new test_df into test_data.csv successfully!")
print("Updated test size:", len(updated_test_df))


Appended new test_df into test_data.csv successfully!
Updated test size: 887


In [ ]:
model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Using device:", device)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using device: cuda


In [ ]:
max_input = 512
max_target = 128

class ReplyDataset(Dataset):
    def __init__(self, df):
        self.inputs = df["model_input"].tolist()
        self.targets = df["model_target"].tolist()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        src = self.inputs[idx]
        tgt = self.targets[idx]

        src_enc = tokenizer(
            src,
            max_length=max_input,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        tgt_enc = tokenizer(
            tgt,
            max_length=max_target,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "input_ids": src_enc["input_ids"].squeeze(),
            "attention_mask": src_enc["attention_mask"].squeeze(),
            "labels": tgt_enc["input_ids"].squeeze()
        }


In [ ]:
train_dataset = ReplyDataset(train_df)
val_dataset   = ReplyDataset(val_df)
test_dataset  = ReplyDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=4, shuffle=False)

len(train_loader), len(val_loader), len(test_loader)

(771, 165, 166)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

In [ ]:
epochs = 3
model.train()

for epoch in range(epochs):
    print(f"\n===== EPOCH {epoch+1}/{epochs} =====")

    # --------- TRAINING ----------
    total_train_loss = 0.0

    for batch in tqdm(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss:.4f}")

    # --------- VALIDATION ----------
    model.eval()
    total_val_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention,
                labels=labels
            )

            total_val_loss += outputs.loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    model.train()


===== EPOCH 1/3 =====


100%|██████████| 771/771 [02:43<00:00,  4.71it/s]


Training Loss: 0.5726
Validation Loss: 0.0311

===== EPOCH 2/3 =====


100%|██████████| 771/771 [02:34<00:00,  4.98it/s]


Training Loss: 0.0756
Validation Loss: 0.0187

===== EPOCH 3/3 =====


100%|██████████| 771/771 [02:37<00:00,  4.91it/s]


Training Loss: 0.0463
Validation Loss: 0.0151


In [ ]:
model.eval()
total_test_loss = 0.0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention,
            labels=labels
        )

        total_test_loss += outputs.loss.item()

avg_test_loss = total_test_loss / len(test_loader)
print(f"\nFinal Test Loss: {avg_test_loss:.4f}")


Final Test Loss: 0.0439


In [ ]:
save_path = "pos_short_generator_retrieval"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("POS_SHORT retrieval-augmented generator saved at:", save_path)

POS_SHORT retrieval-augmented generator saved at: pos_short_generator_retrieval


In [ ]:
def generate_reply(review_text, top_k=3, max_len=150):
    # 1) get retrieval context for this new review
    retrieved_replies = retriever.retrieve(review_text, top_k=top_k)
    context = " ".join(retrieved_replies)

    # 2) build input in same format as training
    final_input = "review: " + review_text + " retrieved: " + context

    # 3) encode and generate
    inputs = tokenizer(final_input, return_tensors="pt", truncation=True, max_length=max_input).to(device)
    output = model.generate(
        **inputs,
        max_length=max_len,
        num_beams=5,
        early_stopping=True
    )

    reply = tokenizer.decode(output[0], skip_special_tokens=True)
    return reply

In [ ]:
sample_review = "Exclusively for good accommodation "
generated = generate_reply(sample_review)
print("REVIEW:")
print(sample_review)
print("\nGENERATED REPLY:")
print(generated)


REVIEW:
Exclusively for good accommodation 

GENERATED REPLY:
Dear Guest, We are truly honoured to be able to host you during your visit to PERSON NAME. It is indeed a pleasure to note that you liked our HOTEL_NAME> its appearance and location. You appreciated the food quality but felt it was expensive. We use good quality of materials for cooking food to maintain quality and quantity. Your feedback with regard to this issue has been noted. We look forward for more opportunities to be of service to you at our HOTEL_NAME>.


In [ ]:
!cp -r /content/pos_short_generator_retrieval "/content/drive/My Drive/"

cp: cannot create directory '/content/drive/My Drive/': No such file or directory
